# 1. 데이터 수집 - 기간 설정해서 네이버 지식인 크롤링 수행

In [ ]:
# 필요한 패키지
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib import request as urlreq
import time
import re
from tqdm import tqdm_notebook
from openpyxl import load_workbook 
import openpyxl as xl


# 키워드, 기간설정, 페이지 번호에 따른 url 가져오는 함수 생성
def mk_url(keyword, from_date, to_date, page_num):
    
    #최신순으로 정렬한 path
    path = 'https://kin.naver.com/search/list.naver?sort=date&query='
    
    period = '&period=' + from_date + '.%7C' + to_date
    url = path + keyword + period + '.&section=qna&page=' + page_num
    
    return url

# 기간설정을 위한 날짜 만드는 함수 생성
def mk_date(month, day):
    f = '2022.%s.%s'%(month.zfill(2), day.zfill(2))
    t = '2022.%s.%s'%(month.zfill(2), day.zfill(2))
    return f, t


############키워드, 날짜 설정하기 위한 변수 설정 부분############
# 한달마다 기간설정하여 크롤링 진행
keyword = '우울증'
month = 3
s_day = 1
e_day = 31
################################################################


date_info = str(month).zfill(2) + str(s_day).zfill(2) + '_' + str(month).zfill(2) + str(e_day).zfill(2) 
f_name = keyword + '_' + date_info +"_crawling_result.xlsx"

#엑셀로 저장하기 위한 변수 설정
wb = xl.Workbook()
sheet = wb.active
sheet.append(['date','user_name','title','content','tag'])


# 크롤링 수행
for i in tqdm_notebook(range(s_day, e_day + 1), desc = '페이지 수집중...'):
    f, t = mk_date(str(month), str(i))
    post_number = None
    current_number = None
    total_number = None
    page_num = 1
    page_url = []
    
    # 1차 크롤링 - 페이지별로 게시글 url 수집
    while True:
        #최신순으로 크롤링
        
        # 크롬 드라이버
        binary = "C:\\data\\chromedriver_win32\\chromedriver.exe"

        # 브라우저를 인스턴스화
        driver = webdriver.Chrome(binary)

        kwd_url = mk_url(keyword,f,t,str(page_num))
        driver.get(kwd_url)
        time.sleep(0.5)
        html = driver.page_source

        soup = BeautifulSoup(html,'html.parser')
        base = soup.find_all('a',class_="_nclicks:qna.txt _searchListTitleAnchor")

        # 개별 주소 가져오기
        for b_num in base:
            page_url.append(b_num.get('href')) 

        # 페이지 넘기기
        post_number = driver.find_element_by_class_name('number').text
        
        current_number = post_number[post_number.find('-')+1:post_number.find('/')]
        current_number = re.sub(',','', current_number)
        
        total_number = post_number[post_number.find('/')+1:post_number.find(')')]
        total_number = re.sub(',','', total_number)
        
        if int(current_number) == int(total_number):
            break
            
        else:
            page_num += 1
            time.sleep(0.5)
        
    page_url = list(set(page_url))  # 페이지 url 중복제거

    # 2차 크롤링 - 각 페이지 게시내용 크롤링
    for i in tqdm_notebook(page_url, desc = '게시글 수집중....'):
        try:
            driver.get(i)
            time.sleep(0.5)
            
            #upload_date
            upload_date = driver.find_element_by_class_name("c-userinfo__info").text
            upload_date = upload_date[upload_date.find('\n')+1:]

            #user name
            user_name = driver.find_element_by_class_name("c-userinfo__author").text
            user_name = user_name[user_name.find('\n')+1:]

            #제목
            title = driver.find_element_by_class_name('title').text

            #내용
            try:
                question_txt = driver.find_element_by_class_name('c-heading__content').text
            except:
                question_txt = ""

            question_txt = re.sub('[\n\r\t]+','',question_txt)

            #태그
            tag = driver.find_element_by_class_name("tag-list__item.tag-list__item--category").text
            tag = tag[tag.find('Ξ')+2:]

            # 엑셀에 게시날짜, 제목, 내용, 태그 저장
            sheet.append([upload_date, user_name, title, question_txt, tag])
            time.sleep(0.5)

        except:
            continue
    

    driver.quit()
    
wb.save("c:\\naver_qna\\" + f_name)  # 엑셀로 저장


# 2. 데이터 전처리 
1) 크롤링된 데이터에서 수집기간에 맞지 않은 데이터들을 제외하고 하나의 데이터 프레임으로 병합하였음

In [ ]:
import pandas as pd
import openpyxl

doc1 = pd.read_excel("c:\\naver_qna\\우울증_0101_0115_crawling_result.xlsx")
doc2 = pd.read_excel("c:\\naver_qna\\우울증_0116_0131_crawling_result.xlsx")
doc3 = pd.read_excel("c:\\naver_qna\\우울증_0201_0228_crawling_result.xlsx")
doc4 = pd.read_excel("c:\\naver_qna\\우울증_0301_0331_crawling_result.xlsx")
doc5 = pd.read_excel("c:\\naver_qna\\우울증_0401_0415_crawling_result.xlsx")
doc6 = pd.read_excel("c:\\naver_qna\\우울증_0416_0430_crawling_result.xlsx")


doc1.date = pd.to_datetime(doc1.date)
doc2.date = pd.to_datetime(doc2.date)
doc3.date = pd.to_datetime(doc3.date)
doc4.date = pd.to_datetime(doc4.date)
doc5.date = pd.to_datetime(doc5.date)
doc6.date = pd.to_datetime(doc6.date)


doc1 = doc1[(doc1.date >= '2022-01-01')&(doc1.date <= '2022-01-15')]
doc2 = doc2[(doc2.date >= '2022-01-16')&(doc2.date <= '2022-01-31')]
doc3 = doc3[(doc3.date >= '2022-02-01')&(doc3.date <= '2022-02-28')]
doc4 = doc4[(doc4.date >= '2022-03-01')&(doc4.date <= '2022-03-31')]
doc5 = doc5[(doc5.date >= '2022-04-01')&(doc5.date <= '2022-04-15')]
doc6 = doc6[(doc6.date >= '2022-04-16')&(doc6.date <= '2022-04-30')]


result = pd.concat([doc1,doc2,doc3,doc4,doc5,doc6]).reset_index()
result = result.iloc[:,1:]
result

2) 우울, 불안, 부정과 관련없는 데이터를 제거하기 위해 임의로 우울 기준 단어를 설정.  
기준에 맞지 않는 글을을 제거하였음.

In [ ]:
result2 = result.drop_duplicates()
result2

word = ['우울','불안','슬퍼','힘든','힘들','죽고','살고','살아야','스트레스','무기력','자살','ㅈㅏㅅㅏㄹ','공황','병원',
       '위로','분노','자존감','조울','상담','치료','눈물','외로','감정','살려','걱정','아파요','불면증','싫어요','죽어도',
       '트라우마','상처', '방황','중독','싫어','울고', '죽으면','폭력', '극단적']

ex_word = result2

for i in word:
    ex_word = ex_word[:][ex_word.content.apply(lambda x: str(x).count(i) == 0 ) ]
    ex_word = ex_word[:][ex_word.title.apply(lambda x: str(x).count(i) == 0 ) ]

ex_word

data_raw = result2.drop(ex_word.index)  


3) 문자열 정제  
한글과 공백이 아닌 문자열은 모두 제거하였음.

In [ ]:
data = data_raw
data.content = data.content.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
data.title = data.title.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

4) 토큰화 
Konlpy의 okt로 텍스트를 토큰화하였음.
- 명사만 추출
- 품사태깅하여 명사, 형용사, 동사만 추출

In [ ]:
# 명사만 추출
from tqdm import tqdm

noun = []
data.content = data.content.astype('str')
for sentence in tqdm(data.content):
    sentence = okt.normalize(sentence)
    noun.append(okt.nouns(sentence))  # 형태소 추출

In [ ]:
from collections import Counter

noun_list = []
for i in noun:
    for k in i:
        if len(k) > 1:
            noun_list.append(k)
noun_list

c = Counter(noun_list)

In [ ]:
# 작성내용에서 텍스트 명사로만 추출하여 워드클라우드 생성

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
from PIL import Image
import numpy as np

img = Image.open('c:\\naver_qna\\head.png')
img_array = np.array(img)

wc = WordCloud(background_color ='white',colormap='gist_earth',
               width = 700, height = 700, font_path='H2GTRE',
               max_font_size=300, max_words=150, mask = img_array)

gen = wc.generate_from_frequencies(c)


plt.figure(figsize = (6,6))
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드
plt.imshow(gen)
plt.title('[작성내용 단어 빈도에 따른 워드클라우드]\n', size = 20)
plt.axis('off')
plt.show()

wc.to_file('c:\\naver_qna\\qna_wordcloud_con.jpg')

In [ ]:
# 작성제목에서 텍스트 명사로만 추출하여 워드클라우드 생성
from tqdm import tqdm

noun = []
data.title = data.title.astype('str')
for sentence in tqdm(data.title):
    sentence = okt.normalize(sentence)
    noun.append(okt.nouns(sentence))  # 형태소 추출

from collections import Counter

noun_list = []
for i in noun:
    for k in i:
        if len(k) > 1:
            noun_list.append(k)
noun_list

c = Counter(noun_list)
df = pd.DataFrame(c.most_common(10), columns = ['word', 'count'])
df.index = df.index + 1
df

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
from PIL import Image
import numpy as np

img = Image.open('c:\\naver_qna\\head.png')
img_array = np.array(img)

wc = WordCloud(background_color ='white',colormap='gist_heat',
               width = 700, height = 700, font_path='H2GTRE',
               max_font_size=300, max_words=150, mask = img_array)

gen = wc.generate_from_frequencies(c)
plt.figure(figsize = (6,6))
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드
plt.imshow(gen)
plt.title('[작성제목 단어 빈도에 따른 워드클라우드]\n', size = 20)
plt.axis('off')
plt.show()

wc.to_file('c:\\naver_qna\\qna_wordcloud_tit.jpg')

# 3. word2vec 진행

In [ ]:
#word2vec 

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from matplotlib import font_manager as fm
from matplotlib import rc

model = Word2Vec(contents, window = 2, min_count = 10)
model

from gensim.models import KeyedVectors
model.wv.save_word2vec_format('c:\\naver_qna\\w2v_contents_model')   #모델 저장

model.wv.vectors.shape

# 4. 단어 유사성 분석

In [ ]:
df1 = pd.DataFrame(model.wv.most_similar('우울증'), columns = [['우울증','우울증'],['word', 'similarity']])
df2 = pd.DataFrame(model.wv.most_similar('생각'), columns = [['생각','생각'],['word', 'similarity']])
df3 = pd.DataFrame(model.wv.most_similar('사람'), columns = [['사람','사람'],['word', 'similarity']])
df4 = pd.DataFrame(model.wv.most_similar('엄마'), columns = [['엄마','엄마'],['word', 'similarity']])

sim_df = pd.DataFrame(pd.concat([df1, df2, df3, df4], axis = 1))
sim_df

# 5. 차원축소 및 군집분석

In [ ]:
tsne = TSNE(n_components = 2)
tsne

vocab = model.wv.vocab
similarity = model[vocab]
similarity #(11420, 100)

import pandas as pd

transform_similarity = tsne.fit_transform(similarity) # 2차원으로 변환됨
con_df = pd.DataFrame(transform_similarity, index = vocab, columns = ['x','y'])
con_df[0:10]

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 4)
predict = kmeans.fit_predict(con_df)
predict

km_result = con_df
km_result['predict'] = predict

import seaborn as sns
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드
plt.rcParams['axes.unicode_minus'] = False # 특수문자 안깨지는 코드

sns.lmplot('x','y', data = km_result, fit_reg = False, size = 6, hue = 'predict', legend = False)
# plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], c = 'red', marker = '*', s = 50, label = '클러스터 중심')

plt.legend(['군집1','군집2','군집3','군집4'])
plt.show()

##################################################

# 클러스터 중심점 찍기
import seaborn as sns
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드

sns.lmplot('x','y', data = km_result, fit_reg = False, size = 6, hue = 'predict', legend = False,
          palette = ['lightsteelblue', 'bisque','darkseagreen','lightpink'])
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], c = 'red', marker = '*', s = 50, label = '클러스터 중심')

plt.legend(['군집1','군집2','군집3','군집4','클러스터 중심'])
plt.show()

In [ ]:
# 군집1
import seaborn as sns
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드

sns.lmplot('x','y', data = km_result, fit_reg = False, size = 6, hue = 'predict', legend = False,
          palette = ['lightsteelblue', 'lightgray','lightgray','lightgray'])
plt.scatter(kmeans.cluster_centers_[0,0], kmeans.cluster_centers_[0,1], c = 'red', marker = '*', s = 50, label = '클러스터 중심')

plt.title('[군집 1]\n', size = 20)
plt.show()

# 군집2
import seaborn as sns
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드

sns.lmplot('x','y', data = km_result, fit_reg = False, size = 6, hue = 'predict', legend = False,
          palette = ['lightgray', 'bisque','lightgray','lightgray'])
plt.scatter(kmeans.cluster_centers_[1,0], kmeans.cluster_centers_[1,1], c = 'red', marker = '*', s = 50, label = '클러스터 중심')

plt.title('[군집 2]\n', size = 20)
plt.show()

# 군집3
import seaborn as sns
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드

sns.lmplot('x','y', data = km_result, fit_reg = False, size = 6, hue = 'predict', legend = False,
          palette = ['lightgray', 'lightgray','darkseagreen','lightgray'])
plt.scatter(kmeans.cluster_centers_[2,0], kmeans.cluster_centers_[2,1], c = 'red', marker = '*', s = 50, label = '클러스터 중심')

plt.title('[군집 3]\n', size = 20)
plt.show()

# 군집4
import seaborn as sns
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드

sns.lmplot('x','y', data = km_result, fit_reg = False, size = 6, hue = 'predict', legend = False,
          palette = ['lightgray', 'lightgray','lightgray','lightpink'])
plt.scatter(kmeans.cluster_centers_[3,0], kmeans.cluster_centers_[3,1], c = 'red', marker = '*', s = 50, label = '클러스터 중심')

plt.title('[군집 4]\n', size = 20)
plt.show()

In [ ]:
import math

cluster1 = km_df[(km_df.predict == 0 )&(km_df.word.apply(lambda x: len(x) > 1))]

dist_cluster1 = []
for i,k in zip(cluster1['x'], cluster1['y']): 
    dist_cluster1.append(math.sqrt(abs(i - kmeans.cluster_centers_[0,0])**2 + abs(k - kmeans.cluster_centers_[0,1])**2))

cluster1['dist_center'] = dist_cluster1
cluster1

cluster1 = cluster1.sort_values(by = 'dist_center', ascending = True).head(10)

##############################################

cluster2 = km_df[(km_df.predict == 1 )&(km_df.word.apply(lambda x: len(x) > 1))]

dist_cluster2 = []
for i,k in zip(cluster2['x'], cluster2['y']): 
    dist_cluster2.append(math.sqrt(abs(i - kmeans.cluster_centers_[1,0])**2 + abs(k - kmeans.cluster_centers_[1,1])**2))

cluster2['dist_center'] = dist_cluster2
cluster2

cluster2 = cluster2.sort_values(by = 'dist_center', ascending = True).head(10)

##############################################

cluster3 = km_df[(km_df.predict == 2 )&(km_df.word.apply(lambda x: len(x) > 1))]

dist_cluster3 = []
for i,k in zip(cluster3['x'], cluster3['y']): 
    dist_cluster3.append(math.sqrt(abs(i - kmeans.cluster_centers_[2,0])**2 + abs(k - kmeans.cluster_centers_[2,1])**2))

cluster3['dist_center'] = dist_cluster3
cluster3

cluster3 = cluster3.sort_values(by = 'dist_center', ascending = True).head(10)

##############################################

cluster4 = km_df[(km_df.predict == 3 )&(km_df.word.apply(lambda x: len(x) > 1))]

dist_cluster4 = []
for i,k in zip(cluster4['x'], cluster4['y']): 
    dist_cluster4.append(math.sqrt(abs(i - kmeans.cluster_centers_[3,0])**2 + abs(k - kmeans.cluster_centers_[3,1])**2))

cluster4['dist_center'] = dist_cluster4
cluster4

cluster4 = cluster4.sort_values(by = 'dist_center', ascending = True).head(10)

In [ ]:
a = pd.DataFrame(pd.concat([cluster1,
                            cluster2,
                            cluster3,
                            cluster4], axis = 1))
a.columns =  [['군집1','군집1','군집2','군집2','군집3','군집3','군집4','군집4'],
           ['word','cnt','word','cnt','word','cnt','word','cnt']]
a